# EEG Analysis

I follow Melara, Wang, Vu, & Proctor (2008) as reference.

In [1]:
import re
import os
os.chdir('../data')

import pandas as pd
import pingouin as pg

In [81]:
with open('input/eeg/N200_Simon_consistent.txt', 'r') as txt:
    consistent = pd.read_csv(txt, sep='[ ]+', engine='python')  # 'python' engine supports regex
with open('input/eeg/N200_Simon_inconsistent.txt', 'r') as txt:
    inconsistent = pd.read_csv(txt, sep='[ ]+', engine='python')

In [82]:
electrodes_con = [name for name in consistent.columns if re.search('[F|C|FC]z', name)]
electrodes_incon = [name for name in inconsistent.columns if re.search('[F|C|FC]z', name)]
electrodes_con

['Fz-BL_Simon_con', 'Cz-BL_Simon_con', 'FCz-BL_Simon_con']

In [83]:
# concat the two dataframes
simon = pd.concat([consistent[electrodes_con], inconsistent[electrodes_incon]], axis=1)
simon

,Fz-BL_Simon_con,Cz-BL_Simon_con,FCz-BL_Simon_con,Fz-BL_Simon_incon,Cz-BL_Simon_incon,FCz-BL_Simon_incon
0,"-2,837336","1,063387","-1,611021","-1,352624","1,534315","-0,374006"
1,"-0,509586","2,769833","1,040470","0,047176","1,605703","0,468330"
2,"2,051602","2,123461","2,348540","0,008106","-0,843206","-0,725597"
3,"1,710116","6,434400","4,068703","1,587750","6,687336","4,168632"
4,"0,426309","0,542873","0,252751","-1,078918","-0,973850","-2,052961"
5,"-0,593096","-1,277104","-0,792954","-0,590188","-1,013159","-0,727562"
6,"0,811415","-4,826107","-1,448437","0,428882","-4,596034","-1,153903"
7,"-1,371791","2,054905","0,645081","-1,676434","1,214297","0,040007"
8,"-2,412884","3,043248","-1,376676","-3,441025","2,748364","-2,113057"
9,"1,312028","5,530357","3,336475","1,439258","4,551743","2,726218"


In [95]:
# make dataframe numerical
for col in simon.columns:
    simon[col] = pd.to_numeric(simon[col].str.replace(',', '.'))

In [97]:
simon.mean()

Fz-BL_Simon_con      -0.280873
Cz-BL_Simon_con       1.704803
FCz-BL_Simon_con      0.557500
Fz-BL_Simon_incon    -0.484569
Cz-BL_Simon_incon     1.065308
FCz-BL_Simon_incon    0.012873
dtype: float64

In [120]:
Fz = pg.ttest(simon['Fz-BL_Simon_con'], simon['Fz-BL_Simon_incon'], alternative='greater', paired=True)
Fz.index = ['Fz']

In [122]:
FCz = pg.ttest(simon['FCz-BL_Simon_con'], simon['FCz-BL_Simon_incon'], alternative='greater', paired=True)
FCz.index = ['FCz']

In [121]:
Cz = pg.ttest(simon['Cz-BL_Simon_con'], simon['Cz-BL_Simon_incon'], alternative='greater', paired=True)
Cz.index = ['Cz']

In [127]:
pd.concat([Fz, FCz, Cz]).drop(columns=["alternative"])

,T,dof,p-val,CI95%,cohen-d,BF10,power
Fz,0.650807,10,0.264920,"[-0.36, inf]",0.130556,0.713,0.107407
FCz,1.485299,10,0.084146,"[-0.12, inf]",0.281392,1.413,0.219457
Cz,2.083925,10,0.031888,"[0.08, inf]",0.210224,2.842,0.160116
